## Dogtor dog's temporal COVID19 visualization solution

<img src="https://s14-eu5.startpage.com/cgi-bin/serveimage?url=https:%2F%2Fsteamuserimages-a.akamaihd.net%2Fugc%2F272841093784221615%2F948967DA5537466A9509113E12C237641EB1506D%2F&sp=9ce3481ad1247d4f63b90d00de600702" alt="The Man himself" style="width:250px;height:200px;">
<audio autoplay>
  <source src="https://a.tumblr.com/tumblr_m9cqm4IkM31qkv3fno1.mp3" type="audio/mpeg"></audio>

This is a 2D visualization solution for temporal COVID19 epidemiological data available from: <strike><a href="https://www.acdc.com/">acdc</a></strike>, <a href="https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide">ecdc</a>.

In [ ]:
from __future__ import print_function
import ipywidgets as widgets
import pandas as pd
import numpy as np
from ipywidgets import interact, interactive, fixed, interact_manual
import plotly.graph_objects as go
from IPython.display import display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
%matplotlib inline

#fetching the data
while True:
    try:
        df = pd.read_csv("https://opendata.ecdc.europa.eu/covid19/casedistribution/csv") 
        # below is a link to an old source
        #("https://www.ecdc.europa.eu/sites/default/files/documents/COVID-19-geographic-disbtribution-worldwide-"+ str(date.today())[:8] + str(int(str(date.today())[8:])-1) + ".xlsx")
        break
    except:
        print("Error fetching data")

# function for fetching the requested countrydata
def preprocess(df, countrylist, days):

    dflist = []
    for country in countrylist:
        dfcountry = df[["dateRep","cases", "deaths", "popData2019"]][df["countriesAndTerritories"] == country].head(days)
        dfcountry['country'] = country
        dfcountry = dfcountry.iloc[::-1]
        dfcountry["cumCases"] = dfcountry["cases"].cumsum(axis = 0, skipna = True)
        dfcountry["cumDeaths"] = dfcountry["deaths"].cumsum(axis = 0, skipna = True)
        dflist.append(dfcountry)
    dfTemporal = pd.concat(dflist)
    dfTemporal["dateRep"] = dfTemporal["dateRep"].astype("str")
    return(dfTemporal)

# function for plotting cases and deaths
def lineplotCaseDeath(df, countrylist, days, popRel):
    
    dates = df["dateRep"].tail(days)
    
    figCases = go.Figure()
    figDeaths = go.Figure()
    
    for country in countrylist:    
        if popRel is False:
            y = df["cumCases"][df["country"] == country]
        else:
            y = (df["cumCases"]/df["popData2019"])[df["country"] == country]
        
        figCases.add_trace(go.Scatter(x=dates, y=y,
                            mode='lines+markers',
                            name=country + ' cases'))

        if popRel is False:
            y = df["cumDeaths"][df["country"] == country]
        else:
            y = (df["cumDeaths"]/df["popData2019"])[df["country"] == country]
        
        figDeaths.add_trace(go.Scatter(x=dates, y=y,
                            mode='lines+markers',
                            name=country + ' deaths'))

    figCases.update_layout(title="Cases:" + str(list(df["country"].unique())))
    figCases.show()

    figDeaths.update_layout(title="Deaths:" + str(list(df["country"].unique())))
    figDeaths.show()

countrylist = []

def plotti(days, countries, popRel):
    global countrylist
    countrylist.append(countries)
    countrylist = list(set(countrylist))
    print(countrylist)
    df2 = preprocess(df, countrylist, days)
    lineplotCaseDeath(df2, countrylist, days, popRel)

def resetCountries(b):
    global countrylist
    countrylist = []
    return countrylist

button = widgets.Button(description="reset countries")

button.on_click(resetCountries)

display(button)

interact_manual(plotti, days = widgets.IntSlider(min=10,max=300,value=30), 
                countries = widgets.Dropdown(options = df["countriesAndTerritories"].unique()), 
                popRel=False)
